In [81]:
import numpy as np
import pandas as pd
from sklearn.calibration import LabelEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.feature_selection import SequentialFeatureSelector
import seaborn as sns
from sklearn.feature_selection import SequentialFeatureSelector
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
import statsmodels.api as sm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping



In [82]:
df1 = pd.read_csv("train.csv")
#df1.head()
df2 = pd.read_csv('test.csv')
#df2

In [ ]:
# Identify and drop duplicate columns
duplicated_columns = df1.columns[df1.columns.duplicated()]
df_no_duplicates = df1.drop(columns=duplicated_columns)

In [ ]:
# Identify and drop duplicate columns
duplicated_columns = df2.columns[df2.columns.duplicated()]
df_no_duplicates = df2.drop(columns=duplicated_columns)

In [83]:
df1 = df1.drop(columns=['sub_area'])
df2 = df2.drop(columns=['sub_area','row ID'])

In [ ]:
df_onehot = pd.get_dummies(df1)
df_onehot.dtypes
test_onehot = pd.get_dummies(df2)

In [ ]:
df_onehot = pd.get_dummies(df1,drop_first=True)
test_onehot = pd.get_dummies(df2,drop_first=True)

In [84]:
categorical_columns = df1.select_dtypes(include=['object']).columns

# Label encode categorical columns
label_encoder = LabelEncoder()
for col in categorical_columns:
   df1[col] = label_encoder.fit_transform(df1[col])
   df2[col] = label_encoder.transform(df2[col])

In [ ]:
X = df_onehot.drop(columns=['price_doc',],axis=1)
y = df_onehot['price_doc']

In [ ]:
y.shape

In [85]:
X = df1.drop(columns=['price_doc',],axis=1)
y = df1['price_doc']

In [ ]:
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)
df2_encoded = imputer.fit_transform(test_onehot)

In [87]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)
df2_standardized = scaler.fit_transform(df2)


In [ ]:
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)
X_testscaled = scaler.fit_transform(test_onehot)

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
df2_encoded_scaled = scaler.fit_transform(df2)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

variance_threshold = 0.001 # Set your desired threshold
selector = VarianceThreshold(threshold=variance_threshold)
X_train_high_variance = selector.fit_transform(X_standardized)
X_test_high_variance = selector.transform(df2_standardized)

In [ ]:
X_train_high_variance.shape


In [ ]:
X_test_high_variance.shape

In [ ]:
#copy y in y_test
y_test = y.copy()

In [ ]:
y_test.shape

In [ ]:
#covert x_scaled to dataframe
X_scaled = pd.DataFrame(X_scaled)
#convert x_testscaled to dataframe
X_testscaled = pd.DataFrame(df2_encoded_scaled)

In [89]:
#covert x_scaled to dataframe
X_scaled = pd.DataFrame(X_standardized)
#convert x_testscaled to dataframe
X_testscaled = pd.DataFrame(df2_standardized)

In [90]:
import statsmodels.api as sm
import pandas as pd

# Convert scaled_df1 to a DataFrame without specifying columns

# Add a constant term to the feature matrix
X_with_const = sm.add_constant(X_scaled)

# Fit a linear regression model
model = sm.OLS(y, X_with_const).fit()

# Get p-values for each feature
p_values = model.pvalues[1:]  # Exclude the constant term

# Set your desired threshold for p-value
threshold = 0.05

# Filter features based on p-value
selected_features = p_values[p_values < threshold].index

# Display selected features
print("Selected Features:")
print(selected_features)

# Select columns in the DataFrame
X_selected = X_scaled[selected_features]


Selected Features:
Index([  0,   1,   2,   3,   5,   6,   8,   9,  10,  11,
       ...
       248, 250, 251, 252, 253, 256, 259, 265, 266, 269],
      dtype='object', length=174)


In [91]:
#keep those columns in X_testscaled that are in X_selected
X_testscaled = X_testscaled[X_selected.columns]

In [ ]:
X_scaled.shape

In [ ]:
test_onehot.shape

In [92]:
X_selected.shape

(181507, 174)

In [93]:
X_testscaled.shape

(77789, 174)

In [ ]:
kBest = SelectKBest(k=20, score_func=f_regression)
X_train_kBest = kBest.fit_transform(X_train_high_variance, y)
X_test_kBest = kBest.transform(X_test_high_variance)

In [ ]:
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train_kBest)
X_test_pca = pca.transform(X_test_kBest)

In [94]:
# get the number of input features in X and assign to n_features
n_features = X_selected.shape[1]
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Create a Sequential model
model = Sequential()

# Add the first hidden layer with 150 neurons and specify the input shape
model.add(Dense(128, input_dim=n_features, activation='relu'))

# Add the second hidden layer with 25 neurons
model.add(Dense(64, activation='relu'))

# Add the output layer with 1 neuron (for binary classification) and 'sigmoid' activation
model.add(Dense(1, activation='sigmoid'))


# Print the model summary
print(model.summary())


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 128)               22400     
                                                                 
 dense_19 (Dense)            (None, 64)                8256      
                                                                 
 dense_20 (Dense)            (None, 1)                 65        
                                                                 
Total params: 30,721
Trainable params: 30,721
Non-trainable params: 0
_________________________________________________________________
None


In [95]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) #change optimizer

In [96]:
model.fit(X_selected, y, epochs=50, batch_size=128, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
1135/1135 [==============================] - 3s 2ms/step - loss: -15739112128512.0000 - accuracy: 0.0000e+00 - val_loss: -56173973733376.0000 - val_accuracy: 0.0000e+00
Epoch 2/50
1135/1135 [==============================] - 2s 2ms/step - loss: -198470438748160.0000 - accuracy: 0.0000e+00 - val_loss: -365391457026048.0000 - val_accuracy: 0.0000e+00
Epoch 3/50
1135/1135 [==============================] - 2s 2ms/step - loss: -720313864159232.0000 - accuracy: 0.0000e+00 - val_loss: -1032758038102016.0000 - val_accuracy: 0.0000e+00
Epoch 4/50
1135/1135 [==============================] - 2s 2ms/step - loss: -1658082190950400.0000 - accuracy: 0.0000e+00 - val_loss: -2108902896304128.0000 - val_accuracy: 0.0000e+00
Epoch 5/50
1135/1135 [==============================] - 3s 2ms/step - loss: -3069097998811136.0000 - accuracy: 0.0000e+00 - val_loss: -3650613753675776.0000 - val_accuracy: 0.0000e+00
Epoch 6/50
1135/1135 [==============================] - 3s 3ms/step - loss: -5000381314

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_high_variance, y)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

In [97]:
md_probs = model.predict(X_testscaled)

2431/2431 [==============================] - 3s 1ms/step


In [ ]:
model.evaluate(testX, testy, batch_size=4096)

In [ ]:
# Evaluate the model on the test set
md_probs = scaler.transform(X_test)  # Don't fit_transform on the test set, use the scaler fitted on the training set
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

In [98]:
additional_data = pd.read_csv("test.csv")

# Flatten md_probs (assuming it's a 2D array)
flat_predictions = md_probs.flatten()

# Create a DataFrame with 'row ID' and predicted values
predicted_df = pd.DataFrame({
    'row ID': additional_data['row ID'],
    'price_doc': flat_predictions  # Replace with the actual column name for your predictions
})

# Save the DataFrame to a new CSV file
predicted_df.to_csv('predictions52.csv', index=False)